In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
#import mca
import math as math
import xgboost as xgb

In [2]:
#Reading in train and test data
trset = pd.read_csv("train.csv")
testv = pd.read_csv('test.csv')

In [3]:
#As explained earlier, dropping the variable v22
trset_nov22 = trset.drop(['v22'],axis=1)
testv_nov22 = testv.drop(['v22'],axis=1)


In [4]:
#Creating the sum of features metric and splitting the data set into two parts
trset_X = trset_nov22.drop(['ID','target'], axis=1)
trsetNaNnum = trset_X.notnull()
trsetNaNnum.astype(int)
trsetNaNnum['TotalFeatures']=trsetNaNnum.sum(axis=1)
trsetNaNnum['ID'] = trset['ID']
trsetNaNnum = trsetNaNnum[['TotalFeatures','ID']]

trsetsum = pd.merge(trset_nov22,trsetNaNnum)

trsetsum27_31 = trsetsum[(trsetsum['TotalFeatures']>25) & (trsetsum['TotalFeatures']<31)]
trsetsum127_131 = trsetsum[(trsetsum['TotalFeatures']>125) & (trsetsum['TotalFeatures']<131)]


In [5]:
#Repeating the operation for the test data
testv_X = testv_nov22.drop(['ID'], axis=1)
tNaNnum = testv_X.notnull()
tNaNnum.astype(int)
tNaNnum['TotalFeatures']=tNaNnum.sum(axis=1)
tNaNnum['ID'] = testv['ID']
tNaNnum = tNaNnum[['TotalFeatures','ID']]

tsum = pd.merge(testv_nov22,tNaNnum)

tsum27_31 = tsum[(tsum['TotalFeatures']>25) & (tsum['TotalFeatures']<31)]
tsum127_131 = tsum[(tsum['TotalFeatures']>125) & (tsum['TotalFeatures']<131)]


In [6]:
tsum27_31.shape

(47628, 132)

In [7]:
#Preparing a list of features to remove from the smaller data set (zero available values)
droplabels=['TotalFeatures']
for label in trsetsum27_31.columns:
    if trsetsum27_31[label].count()==0:
        droplabels.append(label)



In [8]:
#Dopping features
trsetsum27_31=trsetsum27_31.drop(droplabels,axis=1)
trsetsum127_131=trsetsum127_131.drop('TotalFeatures',axis=1)
tsum27_31=tsum27_31.drop(droplabels,axis=1)
tsum127_131=tsum127_131.drop('TotalFeatures',axis=1)


In [9]:
#Figuring out which variables are numeric and which are categorical
t_means = trsetsum27_31.mean()

trset_meandf = t_means.to_frame()

catlabels_sm = [];
alllabels_sm = list(trsetsum27_31.columns.values)
numlabels_sm = list(trset_meandf.index.values)

for label in alllabels_sm:
    if label not in numlabels_sm:
        #print(label)
        catlabels_sm.append(label)

print(catlabels_sm)
len(catlabels_sm)
print(numlabels_sm)
len(numlabels_sm)

['v3', 'v24', 'v30', 'v31', 'v47', 'v52', 'v56', 'v66', 'v71', 'v74', 'v75', 'v79', 'v91', 'v107', 'v110', 'v112', 'v113', 'v125']
['ID', 'target', 'v10', 'v12', 'v14', 'v21', 'v34', 'v38', 'v40', 'v50', 'v62', 'v72', 'v114', 'v129']


14

In [10]:
#Same for the larger data set
t_means = trsetsum127_131.mean()

trset_meandf = t_means.to_frame()

catlabels_lg = [];
alllabels_lg = list(trsetsum127_131.columns.values)
numlabels_lg = list(trset_meandf.index.values)

for label in alllabels_lg:
    if label not in numlabels_lg:
        #print(label)
        catlabels_lg.append(label)

print(catlabels_lg)
len(catlabels_lg)
print(numlabels_lg)
len(numlabels_lg)

['v3', 'v24', 'v30', 'v31', 'v47', 'v52', 'v56', 'v66', 'v71', 'v74', 'v75', 'v79', 'v91', 'v107', 'v110', 'v112', 'v113', 'v125']
['ID', 'target', 'v1', 'v2', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9', 'v10', 'v11', 'v12', 'v13', 'v14', 'v15', 'v16', 'v17', 'v18', 'v19', 'v20', 'v21', 'v23', 'v25', 'v26', 'v27', 'v28', 'v29', 'v32', 'v33', 'v34', 'v35', 'v36', 'v37', 'v38', 'v39', 'v40', 'v41', 'v42', 'v43', 'v44', 'v45', 'v46', 'v48', 'v49', 'v50', 'v51', 'v53', 'v54', 'v55', 'v57', 'v58', 'v59', 'v60', 'v61', 'v62', 'v63', 'v64', 'v65', 'v67', 'v68', 'v69', 'v70', 'v72', 'v73', 'v76', 'v77', 'v78', 'v80', 'v81', 'v82', 'v83', 'v84', 'v85', 'v86', 'v87', 'v88', 'v89', 'v90', 'v92', 'v93', 'v94', 'v95', 'v96', 'v97', 'v98', 'v99', 'v100', 'v101', 'v102', 'v103', 'v104', 'v105', 'v106', 'v108', 'v109', 'v111', 'v114', 'v115', 'v116', 'v117', 'v118', 'v119', 'v120', 'v121', 'v122', 'v123', 'v124', 'v126', 'v127', 'v128', 'v129', 'v130', 'v131']


114

In [11]:
#Combining the unique values of categoricals in both train and test to create a master list for 
#one hot encoding
trset_sm = pd.DataFrame(trsetsum27_31[catlabels_sm])
trset_lg = pd.DataFrame(trsetsum127_131[catlabels_lg])

testv_sm = pd.DataFrame(tsum27_31[catlabels_sm])
testv_lg = pd.DataFrame(tsum127_131[catlabels_lg])

combinedcats_sm = pd.concat([trset_sm,testv_sm])
combinedcats_lg = pd.concat([trset_lg,testv_lg])



In [12]:
#Checking some dimensions
print(trsetsum27_31.shape,tsum27_31.shape)

(47589, 32) (47628, 31)


In [13]:
print(trsetsum127_131.shape,tsum127_131.shape)

(63500, 132) (63534, 131)


In [14]:
combinedcats_sm.shape

(95217, 18)

In [15]:
combinedcats_lg.shape

(127034, 18)

In [16]:
#Creating dummy variables
dums_sm = pd.get_dummies(combinedcats_sm,drop_first=True)

dums_lg = pd.get_dummies(combinedcats_lg,drop_first=True)

In [17]:
#Splitting the dummy variables back into the train and test parts 
dums_sm_tr = dums_sm.iloc[0:len(trsetsum27_31)-1]
dums_sm_test = dums_sm.iloc[len(trsetsum27_31):]

dums_lg_tr = dums_lg.iloc[0:len(trsetsum127_131)-1]
dums_lg_test = dums_lg.iloc[len(trsetsum127_131):]


In [18]:
#Removing the original categoricals and concatenating the dummies back into the main sets
to_drop = catlabels_sm

trset_sm_xgb = pd.concat([trsetsum27_31,dums_sm_tr],axis=1)
trset_sm_xgb = trset_sm_xgb.drop(to_drop, axis=1)

trset_lg_xgb = pd.concat([trsetsum127_131,dums_lg_tr],axis=1)
trset_lg_xgb = trset_lg_xgb.drop(to_drop, axis=1)

testv_sm_xgb = pd.concat([tsum27_31,dums_sm_test],axis=1)
testv_sm_xgb = testv_sm_xgb.drop(to_drop, axis=1)

testv_lg_xgb = pd.concat([tsum127_131,dums_lg_test],axis=1)
testv_lg_xgb = testv_lg_xgb.drop(to_drop, axis=1)



In [19]:
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import log_loss,accuracy_score,roc_auc_score


In [24]:
#The function which is used to generate predictions
def modelfit(alg, dtrain, predictors, test_df, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    #Creating a holdout data set
    dtrain2 = dtrain.sample(frac=1)
    IDS = pd.Series(dtrain2.ID)
    
    splitindex = int(0.3*len(dtrain2))
    dtrain_holdout_ID = dtrain2[:splitindex]
    dtrain_ID = dtrain2[splitindex+1:]
    
    dtrain_holdout = dtrain_holdout_ID.drop('ID',axis=1)
    dtrain = dtrain_ID.drop('ID',axis=1)
    
    #Checking dimensions
    print(dtrain.shape)
    print(dtrain_holdout.shape)
    
    #Using XGB's cross val routine
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[['target']].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='logloss', early_stopping_rounds=early_stopping_rounds, verbose_eval=30)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['target'],eval_metric='logloss')
        
    #Predict test set:
    dtrain_predictions = alg.predict(test_df[predictors])
    dtrain_predprob = alg.predict_proba(test_df[predictors])[:,1]
    
    #Merge the list of IDs with the predicted probabilities, to create a CSV which can be submitted
    sub = {'ID':list(test_df.ID),'PredictedProb':dtrain_predprob}
    subDF = pd.DataFrame(sub)
    
    #Making predictions on the hold out set
    dtrain_predictions = alg.predict(dtrain_holdout[predictors])
    dtrain_predprob = alg.predict_proba(dtrain_holdout[predictors])[:,1]
   
    #Printing some accuracy metrics
    print('Log loss:',log_loss(dtrain_holdout.target,dtrain_predprob))
    print('Accuracy:',accuracy_score(dtrain_holdout.target,np.round(dtrain_predprob).astype(int)))
    print('Area under ROC curve',roc_auc_score(dtrain_holdout.target,dtrain_predprob))
    
    
    #Print model report:
    #print("\nModel Report")
    #print("Accuracy : %.4g" % metrics.accuracy_score(dtrain['target'].values, dtrain_predictions))
    #print("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['target'], dtrain_predprob))
                    
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    #feat_imp.plot(kind='bar', title='Feature Importances')
    #plt.ylabel('Feature Importance Score')
    
    return feat_imp, dtrain_predprob, subDF

In [ ]:

#List of predictors to pass to XGB
predictors_sm = [x for x in trset_sm_xgb.columns if (x != 'target' and x != 'ID')]
predictors_lg = [x for x in trset_lg_xgb.columns if (x != 'target' and x != 'ID')]

trset_sm_run = trset_sm_xgb
trset_lg_run = trset_lg_xgb

#Running a loop for successive feature reduction
for i in range(0,20):
    #Small set classifier
    xgb_sm = XGBClassifier(learning_rate =0.02,n_estimators=500,max_depth=5,min_child_weight=1,
     gamma=0.2,subsample=0.8,colsample_bytree=0.5,objective= 'binary:logistic',
     nthread=8,reg_lambda = 1, max_delta_step=1,seed=i)
    
    #Large set classifier
    xgb_lg = XGBClassifier(learning_rate =0.02,n_estimators=500,max_depth=6,min_child_weight=1,
     gamma=0.1,subsample=0.65,colsample_bytree=0.8,objective= 'binary:logistic',nthread=8,
     reg_lambda =10, max_delta_step=1, seed=i)
    
    print(len(predictors_sm), ' features')
    #Getting list of features by importance, predicted probabilities and the dataframe to write to CSV
    feat_imp_sm, predprob, subDF_sm = modelfit(xgb_sm, trset_sm_run, predictors_sm,testv_sm_xgb)

    print(len(predictors_lg), ' features')
    feat_imp_lg, predprob, subDF_lg = modelfit(xgb_lg, trset_lg_run, predictors_lg,testv_lg_xgb)
    
    #Creating the test CSV
    fname = 'sub'+str(i+1)+'.csv'
    subDF = pd.concat([subDF_sm,subDF_lg])
    subDF.to_csv(fname)
    
    #Getting the top and bottom of the important features list
    featdf = pd.DataFrame(feat_imp_sm)
    featdf.reset_index()
    featdf['newindex']=range(0,len(featdf))
    featdf['vars']=featdf.index
    featdf.set_index('newindex')
    
    #Features to drop
    droplist = list(featdf.vars[-20:])
    #Most important features
    toplist = list(featdf.vars[:50])
    
    print('Small - ',len(predictors_sm), ' top features', predictors_sm )
    
    #Dropping features and relisting the predictors
    trset_sm_run = trset_sm_run.drop(droplist, axis=1)
    predictors_sm = [x for x in trset_sm_run.columns if (x != 'target' and x != 'ID')]

    featdf = pd.DataFrame(feat_imp_lg)
    featdf.reset_index()
    featdf['newindex']=range(0,len(featdf))
    featdf['vars']=featdf.index
    featdf.set_index('newindex')
    
    droplist = list(featdf.vars[-20:])
    toplist = list(featdf.vars[:50])
   
    print('Large - ',len(predictors_lg), ' features', predictors_lg )
    
    
    trset_lg_run = trset_lg_run.drop(droplist, axis=1)
    predictors_lg = [x for x in trset_lg_run.columns if (x != 'target' and x != 'ID')]

    #print('Small:',predictors_sm)
    #print('Large:',predictors_lg)


348  features
(33312, 349)
(14276, 349)
[0]	train-logloss:0.687522+3.53802e-05	test-logloss:0.687596+8.52887e-05
[30]	train-logloss:0.567571+0.00111778	test-logloss:0.57002+0.00223065
[60]	train-logloss:0.507706+0.00131563	test-logloss:0.512702+0.00439044
[90]	train-logloss:0.477995+0.0011938	test-logloss:0.485612+0.00553733
[120]	train-logloss:0.46259+0.00141999	test-logloss:0.472656+0.00606752
[150]	train-logloss:0.453559+0.00141317	test-logloss:0.466107+0.0066154
[180]	train-logloss:0.447309+0.001667	test-logloss:0.462325+0.0068839
[210]	train-logloss:0.442883+0.00174385	test-logloss:0.460216+0.0071
[240]	train-logloss:0.439195+0.00178619	test-logloss:0.458796+0.007307
[270]	train-logloss:0.436294+0.00187077	test-logloss:0.457799+0.00750605
[300]	train-logloss:0.43384+0.00194382	test-logloss:0.457117+0.007677
[330]	train-logloss:0.431565+0.00195788	test-logloss:0.456538+0.00783388
[360]	train-logloss:0.42949+0.0018636	test-logloss:0.456171+0.00797071
[390]	train-logloss:0.427645+0.0